In [ ]:
import subprocess

def install_libraries(lib_list):
    for lib in lib_list:
        try:
            subprocess.run(["pip", "install", lib])
            print(f"Installed {lib} successfully.")
        except Exception as e:
            print(f"Error installing {lib}: {e}")

# List of libraries to install
libraries_to_install = [
    "sys",
    "json",
    "time",
    "ollama",
    "IPython",
    "regex"
]

install_libraries(libraries_to_install)

# Important!!!
if you prompt this ai with something different, make sure to add ```DO NOT USE "``` because if it uses that simbol it will brake the output json braking that key

## WiryNote
if you want easy compatibilty with my other notebook just ask the ai to use this format, make sure to ask for a json
bellow you have a example that worked well for me
```
{ "from": "human", "value": "BLANK" },
{ "from": "gpt", "value": "BLANK" }
```


In [50]:
import sys
import json
import time
import ollama
from IPython.display import clear_output
import re

import os
filess = [
    "output.json",
    "output1.json",
    "output2.json",
    "outputRAWcorrected.json"

]
endfile='usemefortraining.json'

output_file = "output.json"
conversationuwu = """print 10 conversations between Megumin's who is GPT and the human, talk about her characteristics, her body type, height, and age,
You are a young girl named Megumin with a petite build, around 140 cm tall and weighing approximately 35 kg (77 lbs).
She is female, known for her explosive magical abilities, and has a strong-willed and determined personality,
often characterized by her passion for magic and her catchphrase Explosion! DO NOT USE "
human will refer to GPT as you, you are not a gpt chatbot, you are megumin, nobody else. when talking about megumin, you need to talk in first person, you are Megumin
make sure the conversations is typed in json compatible format:
{ "from": "human", "value": "BLANK" },
{ "from": "gpt", "value": "BLANK" }
"""

resetAfterThisCycles= 10 #it will reset the cell output after this many runs, too high and your RAM will regeret it
cl = 0 # set to 1 if you want to delete the cell output after each run, trust me, you'll need it

In [ ]:

def print_memory_usage():
    for var_name, var in globals().items():
        print(f"{var_name}: {sys.getsizeof(var)} bytes")
def escape_double_quotes(json_str):
    parts = json_str.split('"value": "')
    if len(parts) == 2:
        value_part = parts[1].replace('"', '\\"')
        return f'{{"from": "{parts[0].strip()}", "value": "{value_part[:-2].strip()}"}}'
    return None

try:
    cycles = 0
    while True:
        cycles += 1
        input_message = conversationuwu  # Assuming conversationuwu is defined somewhere
        response = ollama.chat(model='example', messages=[{'role': 'user', 'content': input_message}])
        json_string = response['message']['content'] #json string is string
        def extract_json_objects(text):
            json_objects = []
            start = None
            for i, char in enumerate(text):
                if char == '{':
                    start = i
                elif char == '}' and start is not None:
                    json_str = text[start:i+1]
                    try:
                        json_obj = json.loads(json_str)
                        json_objects.append(json_obj)
                    except json.JSONDecodeError:
                        pass
                    start = None
            return json_objects
        jsona=extract_json_objects(json_string)
        with open('output.json', 'r') as file:
            prevjson = json.load(file)
        prevjson.append(jsona)
        with open(filess[0], 'w') as file:
            json.dump(prevjson, file, indent=4, separators=(',', ':'))
        print_memory_usage()
        time.sleep(1)
        if cl == 1 or cycles == resetAfterThisCycles:
            clear_output()  # Clear output after specified cycles
except KeyboardInterrupt:
    print("Loop interrupted.")


In [ ]:


def process_json_file(input_file, output_file):
    # Read the input JSON file
    with open(input_file, 'r') as f:
        data = f.read()
    
    # Remove all instances of '[' and ']'
    cleaned_data = data.replace('[', '').replace(']', '')

    # Add '[' at the beginning and ']' at the end
    final_data = '[' + cleaned_data.strip() + ']'

    # Write the updated JSON data to the output file
    with open(output_file, 'w') as f:
        f.write(final_data)

# Example usage:
input_file = filess[0]  # Replace 'input.json' with your input JSON file path
output_file = filess[1]  # Replace 'output.json' with your output JSON file path
process_json_file(input_file, output_file)


In [ ]:


# Load your JSON data
with open('output1.json', 'r') as file:
    data = json.load(file)

# Filter lines based on the sender and extract "value"
filtered_data = []
for line in data:
    if line.get('from') in ['human', 'gpt'] and 'value' in line:
        filtered_data.append({
            'from': line['from'],
            'value': line['value']
        })

# Save the filtered data to a new JSON file
with open(filess[2], 'w') as outfile:
    json.dump(filtered_data, outfile, indent=4)


In [ ]:


# Load the content of output.json
with open(filess[2], 'r') as f:
    json_data = json.load(f)

# Function to filter out consecutive repeats of "from" and print deleted lines
def filter_consecutive_repeats(data):
    filtered_data = []
    prev_sender = None
    repeat_count = 0
    for item in data:
        sender = item.get("from")
        if sender == prev_sender:
            repeat_count += 1
            if repeat_count <= 1:  # Keep the first occurrence
                filtered_data.append(item)
            else:
                print(f"Deleted line: {item}")  # Print the deleted line
        else:
            filtered_data.append(item)
            repeat_count = 0
        prev_sender = sender
    return filtered_data

# Filter out consecutive repeats in the loaded data and print deleted lines
filtered_data = filter_consecutive_repeats(json_data)

# Save the filtered data to output2.json
with open(endfile, 'w') as f:
    json.dump(filtered_data, f, indent=2)


In [ ]:
import json

# Load input JSON data from file
input_file = filess[3]

with open(input_file, "r") as file:
    input_data = json.load(file)

# Convert conversations to the desired format
output_data = []
current_conversation = None
for item in input_data.get("conversations", []):
    if isinstance(item, dict) and "from" in item and item["from"] == "human":
        if current_conversation is not None:
            output_data.append(current_conversation)
        current_conversation = {"conversations": [item]}
    elif current_conversation is not None:
        current_conversation["conversations"].append(item)

# Add the last conversation
if current_conversation is not None:
    output_data.append(current_conversation)

# Save output data to a new JSON file
output_file = endfile

with open(output_file, "w") as file:
    json.dump(output_data, file, indent=2)

print(f"Converted conversations saved to {output_file}")


In [ ]:
###clean function


def delete_files(file_list):
    for file_name in file_list:
        try:
            os.remove(file_name)
            print(f"Deleted: {file_name}")
        except OSError as e:
            print(f"Error deleting {file_name}: {e}")

# List of files to delete


delete_files(filess)

